# JSON Mini-Project

This Jupyter Notebook contains the solutions for the JSON Mini-Project
which is part of the Springboard curriculum. 

### 1) Find the 10 countries with most projects

We begin by loading all the libraries we will be using, 
and reading the data as a pandas dataframe.

In [30]:
# import necessary libraries
import json
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np

# read in json data
with open("data/world_bank_projects.json") as json_file:
    json_data = json.load(json_file)

# read data as dataframe, look at first 3 lines
world_bank = json_normalize(json_data)
world_bank.head(3)

,_id.$oid,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,docty,envassesmentcategorycode,grantamt,ibrdcommamt,id,idacommamt,impagency,lendinginstr,lendinginstrtype,lendprojectcost,majorsector_percent,mjsector_namecode,mjtheme,mjtheme_namecode,mjthemecode,prodline,prodlinetext,productlinetype,project_abstract.cdata,project_name,projectdocs,projectfinancialtype,projectstatusdisplay,regionname,sector,sector1.Name,sector1.Percent,sector2.Name,sector2.Percent,sector3.Name,sector3.Percent,sector4.Name,sector4.Percent,sector_namecode,sectorcode,source,status,supplementprojectflg,theme1.Name,theme1.Percent,theme_namecode,themecode,totalamt,totalcommamt,url
0,52b213b38594d8a2be17c780,1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,"Project Information Document,Indigenous People...",C,0,0,P129828,130000000,MINISTRY OF EDUCATION,Investment Project Financing,IN,550000000,"[{'Percent': 46, 'Name': 'Education'}, {'Perce...","[{'code': 'EX', 'name': 'Education'}, {'code':...",[Human development],"[{'code': '8', 'name': 'Human development'}, {...","8,11",PE,IBRD/IDA,L,The development objective of the Second Phase ...,Ethiopia General Education Quality Improvement...,"[{'DocDate': '28-AUG-2013', 'EntityID': '09022...",IDA,Active,Africa,"[{'Name': 'Primary education'}, {'Name': 'Seco...",Primary education,46,Secondary education,26.0,Public administration- Other social services,16.0,Tertiary education,12.0,"[{'code': 'EP', 'name': 'Primary education'}, ...","ET,BS,ES,EP",IBRD,Active,N,Education for all,100,"[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,52b213b38594d8a2be17c781,2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,"Project Information Document,Integrated Safegu...",C,4700000,0,P144674,0,MINISTRY OF FINANCE,Specific Investment Loan,IN,5700000,"[{'Percent': 70, 'Name': 'Public Administratio...","[{'code': 'BX', 'name': 'Public Administration...","[Economic management, Social protection and ri...","[{'code': '1', 'name': 'Economic management'},...","1,6",RE,Recipient Executed Activities,L,NaN,TN: DTF Social Protection Reforms Support,"[{'DocDate': '29-MAR-2013', 'EntityID': '00033...",OTHER,Active,Middle East and North Africa,[{'Name': 'Public administration- Other social...,Public administration- Other social services,70,General public administration sector,30.0,NaN,NaN,NaN,NaN,"[{'code': 'BS', 'name': 'Public administration...","BZ,BS",IBRD,Active,N,Other economic management,30,"[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en
2,52b213b38594d8a2be17c782,2014,November,2013-11-01T00:00:00Z,MINISTRY OF FINANCE AND ECONOMIC DEVEL,NaN,Tuvalu!$!TV,TV,Tuvalu,Tuvalu,"Resettlement Plan,Environmental Assessment,Int...",B,0,0,P145310,6060000,MINISTRY OF TRANSPORT AND COMMUNICATIONS,Investment Project Financing,IN,6060000,"[{'Percent': 100, 'Name': 'Transportation'}]","[{'code': 'TX', 'name': 'Transportation'}]","[Trade and integration, Public sector governan...","[{'code': '5', 'name': 'Trade and integration'...","5,2,11,6",PE,IBRD/IDA,L,NaN,Tuvalu Aviation Investment Project - Additiona...,"[{'DocDate': '21-OCT-2013', 'EntityID': '00033...",IDA,Active,East Asia and Pacific,[{'Name': 'Rural and Inter-Urban Roads and Hig...,Rural and Inter-Urban Roads and Highways,100,NaN,NaN,NaN,NaN,NaN,NaN,"[{'code': 'TI', 'name': 'Rural and Inter-Urban...",TI,IBRD,Active,Y,Regional integration,46,"[{'code': '47', 'name': 'Regional integration'...","52,81,25,47",6060000,6060000,http://www.worldbank.org/projects/P145310?lang=en


Next we'll count how many projects each country had, list the countries in descending order, and find the top ten.

In [31]:
# group data and count number of projects by country, display first 10
world_bank.groupby("countryshortname").size().sort_values(ascending = False).head(10)

countryshortname
Indonesia             19
China                 19
Vietnam               17
India                 16
Yemen, Republic of    13
Nepal                 12
Bangladesh            12
Morocco               12
Mozambique            11
Africa                11
dtype: int64

### 2) Find the top 10 major project themes (using column 'mjtheme_namecode')

Now we wish to find the top ten most common themes. This information is nested within the 'mjtheme_namecode' column. So we will need to extract the data into a new dataframe, and find the top ten themes.

In [32]:
# use json_normalize to access nested data
theme = json_normalize(data = json_data, record_path = "mjtheme_namecode")
theme.head(5)

,code,name
0,8,Human development
1,11,
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration


We can see we may have some missing data in the 'name' column, but we'll worry about this in the next problem. For now we'll go ahead and group the data by theme name, count how many there are, and display the top ten.

In [33]:
# group by theme name, display top ten most common themes
theme.groupby("name").size().sort_values(ascending = False).head(10)

name
Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
dtype: int64

### 3) Create a dataframe with the missing names filled in.

The above result doesn't make much sense, the theme with no name has 122 entries. So now we'll deal with those missing names, and then go back and update our previous result.

In [34]:
# create new dataframe 
df = theme.sort_values(by = "code", ascending = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1499 entries, 458 to 1102
Data columns (total 2 columns):
code    1499 non-null object
name    1499 non-null object
dtypes: object(2)
memory usage: 35.1+ KB


No missing values indicates we have empty strings, which we'll replace with NaN values. We can then replace those with the correct strings.

In [37]:
# replace empty string with NaN
df.replace(to_replace = "", value = np.nan, inplace = True)

# confirm we replaced empty strings with NaN
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1499 entries, 458 to 1102
Data columns (total 2 columns):
code    1499 non-null object
name    1499 non-null object
dtypes: object(2)
memory usage: 35.1+ KB


In [39]:
# replace NaN with correct string
df["name"].fillna(inplace = True, method = "bfill")

# check that NaN were replaced
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1499 entries, 458 to 1102
Data columns (total 2 columns):
code    1499 non-null object
name    1499 non-null object
dtypes: object(2)
memory usage: 35.1+ KB


Now that we've replaced the empty names with their correct names, we'll once again find the top ten themes.

In [42]:
# with new df, find top ten themes
df.groupby(["code", "name"]).size().sort_values(ascending = False).head(10)

code  name                                        
11    Environment and natural resources management    250
10    Rural development                               216
8     Human development                               210
2     Public sector governance                        199
6     Social protection and risk management           168
4     Financial and private sector development        146
7     Social dev/gender/inclusion                     130
5     Trade and integration                            77
9     Urban development                                50
1     Economic management                              38
dtype: int64

In [43]:
# double check this is correct by counting the top ten "code" variable
df.groupby("code").size().sort_values(ascending = False).head(10)

code
11    250
10    216
8     210
2     199
6     168
4     146
7     130
5      77
9      50
1      38
dtype: int64